# 5. 마을회관 500m 내 180분 이상 버스정류장 수

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
route=pd.read_csv('dongwan_busRoute.csv', encoding='utf-8')
print("route:\n", route.head(3))

bus_time=pd.read_csv('busTime_all.csv', encoding='utf-8')
print("time:\n", bus_time.head(3))


In [ ]:
busTime_180=bus_time[bus_time['배차간격']>=180]
#print(busTime_180)

In [ ]:
y=pd.DataFrame(route["정류장명"].value_counts())
y.reset_index(inplace=True)

#print(y.head(2))
#print(y['정류장명'].head(3))

In [ ]:
merge_df=pd.merge(busTime_180, route, how='left')
print(type(merge_df))
#print(len(merge_df))
#print(list(merge_df))

In [ ]:
x=pd.DataFrame(merge_df['정류장명'].value_counts())
x.reset_index(inplace=True)
print(x.head(2))

In [ ]:
print(list(y))
print(y['index'].head(3))
print(len(x)); print(len(y))

In [ ]:

x=x.rename({'index':'정류장명', '정류장명':'x_count'}, axis='columns')
print(x.head(3))
y=y.rename({'index':'정류장명', '정류장명':'y_count'}, axis='columns')
print(y.head(3))
    

In [ ]:
merge_xy=pd.merge(x, y, how='left')
print(merge_xy.head(3))

In [ ]:
result=pd.DataFrame(merge_xy[merge_xy['x_count']==merge_xy['y_count']])

In [ ]:
print(result)

In [ ]:
print(result['정류장명'])

In [ ]:
bus_180=pd.merge(result, route, on='정류장명', how='left')

In [ ]:
bus_180=bus_180.drop_duplicates(['정류장명'])
bus_180=bus_180.reset_index(drop=True, inplace=False)

In [ ]:
print(bus_180)

In [ ]:
senior_df=pd.read_csv('final_senior_center.csv', encoding='utf-8') # 경로당 데이터
#print(senior_df.head(5))

jeonju_df=senior_df["borough"].str.contains("전주시") # 전주시만 추출

df=senior_df[jeonju_df]
#print(df["address"].tail(2))

In [ ]:
bus_df=pd.read_csv('busStop.csv', encoding='utf-8') # 버스정류장 데이터
#bus_df.head(2)

In [ ]:
!pip install import_ipynb
import import_ipynb
import busStop_distance

GeoUtil=busStop_distance.GeoUtil()


In [ ]:
center_x=df['X'] # 경로당(마을회관) x 좌표
center_y=df['Y'] # 경로당(마을회관) y 좌표

busStop_x=bus_df['X'] # 버스정류장 x 좌표
busStop_y=bus_df['Y'] # 버스정류장 y 좌표

bus_180_x=bus_180['LNG'] # 180분 이상 버정 x
bus_180_y=bus_180['LAT'] # 180분 이상 버정 y

In [ ]:
print(bus_180_x.head(2)); print(bus_180_y.head(2))

print(center_x.head(2)); print(center_y.head(2))

In [ ]:
cnt_all=[] # 경로당에서 버스정류장까지 거리 리스트
cnt_180=[]

for k in range(len(df)): # 리스트 초기화
    cnt_all.append(0)
    cnt_180.append(0)
    
#stop_name=bus_df['busStop']
#stop_name_180=result['정류장명']


for i in range(0, len(df)): # 경로당에서 제일 가까운 버스정류장까지 거리
    for j in range(0, len(bus_df)):
        if (GeoUtil.get_harversion_distance(center_x[i], center_y[i], busStop_x[j], busStop_y[j])) <= 0.5:
            cnt_all[i]+=1
            
    for j in range(0, len(bus_180)):
        if (GeoUtil.get_harversion_distance(center_x[i], center_y[i], bus_180_x[j], bus_180_y[j])) <= 0.5:
            cnt_180[i]+=1
           

In [ ]:
cnt_all=pd.DataFrame(cnt_all)
cnt_all.columns=['busStop_num']

cnt_180=pd.DataFrame(cnt_180)
cnt_180.columns=['busStop_180_num']

In [ ]:
cnt_df=pd.concat([df, cnt_all, cnt_180], axis=1, join='inner')
cnt_df.head(2)

In [ ]:
cnt_df

In [ ]:
cnt_df.to_csv("over180_num.csv", encoding='euc-kr')